<a href="https://colab.research.google.com/github/parmidamardi/CDM-MatrixMonitor-FP/blob/main/CDM%E2%80%91MatrixMonitor%E2%80%91FP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas numpy matplotlib seaborn openpyxl

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# url = "https://raw.githubusercontent.com/parmidamardi/CDM-MatrixMonitor-FP/main/Online%20Retail.xlsx"
# df_2 = pd.read_excel(url, engine='openpyxl')
# print(df_2.columns)

In [3]:
df = pd.read_csv("https://archive.ics.uci.edu/static/public/352/data.csv", encoding='unicode_escape')
print(df.columns)

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')


In [4]:

print(df.shape)
print(df.head())

(541909, 8)
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom  


In [5]:
print(df.columns.tolist())


['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']


In [6]:
#Drop rows where any key business attribute is missing
df_clean = df.dropna(subset=['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country'])


In [7]:
print(df_clean['InvoiceDate'].dtype)

object


In [8]:
#Convert InvoiceDate to datetime type
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'], errors='coerce')

/tmp/ipython-input-1746825974.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'], errors='coerce')


In [9]:
print(df_clean['InvoiceDate'].dtype)

datetime64[ns]


In [10]:
# Filter invalid transactions
df_clean = df_clean[
    (~df_clean['InvoiceNo'].str.startswith('C')) & (df_clean['Quantity'] > 0) & (df_clean['UnitPrice'] > 0)
]

In [11]:
#Reset index for clean iteration
df_clean = df_clean.reset_index(drop=True)

In [12]:
#After cleaning
print("Shape:", df_clean.shape)
print("Columns:", df_clean.columns.tolist())
df_clean.info()
df_clean.describe()
print(df.head(5))

Shape: (397884, 8)
Columns: ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397884 entries, 0 to 397883
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397884 non-null  object        
 1   StockCode    397884 non-null  object        
 2   Description  397884 non-null  object        
 3   Quantity     397884 non-null  int64         
 4   InvoiceDate  397884 non-null  datetime64[ns]
 5   UnitPrice    397884 non-null  float64       
 6   CustomerID   397884 non-null  float64       
 7   Country      397884 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 24.3+ MB
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053        

In [13]:
basket_df = df_clean[['InvoiceNo', 'StockCode', 'Quantity']].copy()
basket_df = basket_df[basket_df['Quantity'] > 0]
basket_df['Quantity'] = 1

In [14]:
item_matrix = basket_df.pivot_table(
    index='InvoiceNo',
    columns='StockCode',
    values='Quantity',
    aggfunc='sum',
    fill_value=0
)
item_matrix = (item_matrix > 0).astype(int)
print(item_matrix.shape)
print(item_matrix.head(15))

(18532, 3665)
StockCode  10002  10080  10120  10123C  10124A  10124G  10125  10133  10135  \
InvoiceNo                                                                     
536365         0      0      0       0       0       0      0      0      0   
536366         0      0      0       0       0       0      0      0      0   
536367         0      0      0       0       0       0      0      0      0   
536368         0      0      0       0       0       0      0      0      0   
536369         0      0      0       0       0       0      0      0      0   
536370         1      0      0       0       0       0      0      0      0   
536371         0      0      0       0       0       0      0      0      0   
536372         0      0      0       0       0       0      0      0      0   
536373         0      0      0       0       0       0      0      0      0   
536374         0      0      0       0       0       0      0      0      0   
536375         0      0      0       0

In [15]:
print(item_matrix.loc["536365"]["85123A"])

1


In [16]:
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'], errors='coerce')
df_clean = df_clean.sort_values('InvoiceDate').reset_index(drop=True)

In [17]:
df_clean['InvoiceDay'] = df_clean['InvoiceDate'].dt.date
print(df_clean[['InvoiceNo', 'InvoiceDate', 'InvoiceDay']].head(5))

  InvoiceNo         InvoiceDate  InvoiceDay
0    536365 2010-12-01 08:26:00  2010-12-01
1    536365 2010-12-01 08:26:00  2010-12-01
2    536365 2010-12-01 08:26:00  2010-12-01
3    536365 2010-12-01 08:26:00  2010-12-01
4    536365 2010-12-01 08:26:00  2010-12-01


In [18]:
time_batches = [group for _, group in df_clean.groupby('InvoiceDay')]

print(f"Total time batches: {len(time_batches)}")

for i, batch in enumerate(time_batches[:3]):
    print(f"Batch {i+1}: {batch['InvoiceDay'].iloc[0]}  ->  {len(batch)} invoices")

Total time batches: 305
Batch 1: 2010-12-01  ->  1942 invoices
Batch 2: 2010-12-02  ->  2000 invoices
Batch 3: 2010-12-03  ->  1103 invoices


In [19]:
invoice_order = (
    df_clean[['InvoiceNo', 'InvoiceDay']]
    .drop_duplicates()
    .sort_values('InvoiceDay')
)
ordered_invoices = invoice_order['InvoiceNo'].tolist()
item_matrix = item_matrix.loc[ordered_invoices]

In [20]:
day_to_invoices = {
    d: g['InvoiceNo'].unique().tolist()
    for d, g in df_clean.groupby('InvoiceDay')
}

stream_batches = [item_matrix.loc[day_to_invoices[d]] for d in day_to_invoices]

print(f"Total batches (days): {len(stream_batches)}")
print(f"Example batch shape: {stream_batches[0].shape}")

Total batches (days): 305
Example batch shape: (121, 3665)


In [22]:
def gaussian_random_projection_sketch(A, k=50):
    n, d = A.shape
    R = np.random.normal(0, 1/np.sqrt(k), size=(d, k))
    B = A @ R
    return B

In [23]:
def incremental_pca_sketch(A, k=50):
    A_centered = A - np.mean(A, axis=0)
    U, S, Vt = np.linalg.svd(A_centered, full_matrices=False)
    B = (U[:, :k] * S[:k]) @ Vt[:k, :]
    return B, Vt[:k, :]

In [24]:
def frequent_directions_sketch(A, l=50, B=None):
    if B is None:
        B = np.zeros((0, A.shape[1]))
    B = np.vstack((B, A))
    if B.shape[0] > l:
        U, S, Vt = np.linalg.svd(B, full_matrices=False)
        delta = S[l//2]**2
        S = np.sqrt(np.maximum(S**2 - delta, 0))
        B = np.diag(S) @ Vt
        B = B[:l, :]
    return B

In [30]:
def frobenius_error(A, B):
    try:
        ATA = A @ A.T
        BTB = B @ B.T
        if ATA.shape != BTB.shape:
            ATA = A.T @ A
            BTB = B.T @ B
        return np.linalg.norm(ATA - BTB, 'fro') / np.linalg.norm(ATA, 'fro')
    except ValueError:
        ATA = A.T @ A
        BTB = B.T @ B
        return np.linalg.norm(ATA - BTB, 'fro') / np.linalg.norm(ATA, 'fro')

def variance_retained(A, B):
    return np.linalg.norm(B, 'fro')**2 / np.linalg.norm(A, 'fro')**2

In [45]:
def reconstruction_error(A, B, method=None, components=None):

    if method == 'GRP':
        A_hat = B @ B.T @ A
    elif method == 'IPCA':
        if components is not None:
            A_centered = A - np.mean(A, axis=0)
            A_hat = (A_centered @ components.T) @ components + np.mean(A, axis=0)
        else:
            A_hat = B
    elif method == 'FD':
        A_hat = A @ (B.T @ B)
    else:
        A_hat = B

    return np.linalg.norm(A - A_hat, 'fro') / np.linalg.norm(A, 'fro')

In [43]:
metrics = {
    "batch": [], "algo": [],
    "fro_err": [], "var_ret": [],
    "recon_err": []
}

k = 50
B_fd = None
V_ipca = None

for i, A_batch in enumerate(stream_batches):
    A = A_batch.values

    # --- GRP ---
    B_grp = gaussian_random_projection_sketch(A, k)
    metrics["batch"].append(i)
    metrics["algo"].append("GRP")
    metrics["fro_err"].append(frobenius_error(A, B_grp))
    metrics["var_ret"].append(variance_retained(A, B_grp))
    metrics["recon_err"].append(reconstruction_error(A, B_grp, method='GRP'))

    # --- IPCA ---
    # B_ipca, V_ipca = incremental_pca_sketch(A, k)
    # metrics["batch"].append(i)
    # metrics["algo"].append("IPCA")
    # metrics["fro_err"].append(frobenius_error(A, B_ipca))
    # metrics["var_ret"].append(variance_retained(A, B_ipca))
    # metrics["recon_err"].append(reconstruction_error(A, B_ipca, method='IPCA', components=V_ipca))

    # # --- FD ---
    # B_fd = frequent_directions_sketch(A, k, B=B_fd)
    # metrics["batch"].append(i)
    # metrics["algo"].append("FD")
    # metrics["fro_err"].append(frobenius_error(A, B_fd))
    # metrics["var_ret"].append(variance_retained(A, B_fd))
    # metrics["recon_err"].append(reconstruction_error(A, B_fd, method='FD'))

In [44]:
metrics = {
    "batch": [], "algo": [],
    "fro_err": [], "var_ret": [],
    "recon_err": []
}

k = 50
B_fd = None
V_ipca = None

for i, A_batch in enumerate(stream_batches):
    A = A_batch.values

    # --- GRP ---
    # B_grp = gaussian_random_projection_sketch(A, k)
    # metrics["batch"].append(i)
    # metrics["algo"].append("GRP")
    # metrics["fro_err"].append(frobenius_error(A, B_grp))
    # metrics["var_ret"].append(variance_retained(A, B_grp))
    # metrics["recon_err"].append(reconstruction_error(A, B_grp, method='GRP'))

    # --- IPCA ---
    B_ipca, V_ipca = incremental_pca_sketch(A, k)
    metrics["batch"].append(i)
    metrics["algo"].append("IPCA")
    metrics["fro_err"].append(frobenius_error(A, B_ipca))
    metrics["var_ret"].append(variance_retained(A, B_ipca))
    metrics["recon_err"].append(reconstruction_error(A, B_ipca, method='IPCA', components=V_ipca))

    # # --- FD ---
    # B_fd = frequent_directions_sketch(A, k, B=B_fd)
    # metrics["batch"].append(i)
    # metrics["algo"].append("FD")
    # metrics["fro_err"].append(frobenius_error(A, B_fd))
    # metrics["var_ret"].append(variance_retained(A, B_fd))
    # metrics["recon_err"].append(reconstruction_error(A, B_fd, method='FD'))

In [46]:
metrics = {
    "batch": [], "algo": [],
    "fro_err": [], "var_ret": [],
    "recon_err": []
}

k = 50
B_fd = None
V_ipca = None

for i, A_batch in enumerate(stream_batches):
    A = A_batch.values

    # --- GRP ---
    # B_grp = gaussian_random_projection_sketch(A, k)
    # metrics["batch"].append(i)
    # metrics["algo"].append("GRP")
    # metrics["fro_err"].append(frobenius_error(A, B_grp))
    # metrics["var_ret"].append(variance_retained(A, B_grp))
    # metrics["recon_err"].append(reconstruction_error(A, B_grp, method='GRP'))

    # --- IPCA ---
    # B_ipca, V_ipca = incremental_pca_sketch(A, k)
    # metrics["batch"].append(i)
    # metrics["algo"].append("IPCA")
    # metrics["fro_err"].append(frobenius_error(A, B_ipca))
    # metrics["var_ret"].append(variance_retained(A, B_ipca))
    # metrics["recon_err"].append(reconstruction_error(A, B_ipca, method='IPCA', components=V_ipca))

    # # --- FD ---
    B_fd = frequent_directions_sketch(A, k, B=B_fd)
    metrics["batch"].append(i)
    metrics["algo"].append("FD")
    metrics["fro_err"].append(frobenius_error(A, B_fd))
    metrics["var_ret"].append(variance_retained(A, B_fd))
    metrics["recon_err"].append(reconstruction_error(A, B_fd, method='FD'))

In [48]:
df_metrics = pd.DataFrame(metrics)
display(df_metrics.head(20))

,batch,algo,fro_err,var_ret,recon_err
0,0,FD,0.529412,0.330119,35.395352
1,1,FD,0.593751,0.390973,69.079228
2,2,FD,1.159523,0.693093,48.144700
3,3,FD,0.716325,0.420890,96.154974
4,4,FD,1.282795,0.513862,94.545243
5,5,FD,2.101360,0.809908,89.281018
6,6,FD,1.423358,0.545052,108.103853
7,7,FD,1.875737,0.594664,106.318575
8,8,FD,2.112277,0.752214,112.831150
9,9,FD,1.823023,0.863738,158.249393
